# Junta os CSV

Junta todos os csv de um determinado diretório em um único csv

In [1]:
import pandas as pd
import os

# Functions

In [2]:
def join_csv(pathFrom, fileTo, field='Fechamento', fielddate='Data', limit=0, invert=False, formatdatedefault=''):
    counter = 0
    newCsv = pd.DataFrame()

    for file in os.listdir(pathFrom):
        counter = counter+1
        csv = pd.read_csv(pathFrom+'/'+file)
        
        # first column is Data
        if(counter == 1):
            if (formatdatedefault == ''):
                newCsv['Data'] = csv[fielddate]
            else:
                newCsv['Data'] = pd.to_datetime(csv[fielddate], format=formatdatedefault).dt.strftime('%d/%m/%Y')

        name = os.path.splitext(file)[0]
        name = name.replace('.SAO', '')
        newCsv[name] = csv[field]
    
    # inverte os dados: do mais antigo para o mais novo
    if (invert):
        newCsv = newCsv.iloc[::-1].copy()
        newCsv.reset_index(drop=True, inplace=True)
            
    if (limit > 0):
        # orderna do mais recente para o mais antigo
        pos = newCsv.shape[0]-limit
        newCsv = newCsv[pos:]

    newCsv.to_csv(fileTo, index=False)
    print('New file CSV successfully created')
    
def find_csv_max_lines(path):
    rows = []
    for file in os.listdir(path):
        with open(path+'/'+file) as f:
            lines = sum(1 for line in f)
            rows.append([file, lines])

    csv = pd.DataFrame(rows, columns=['File', 'Lines'])
    maxlines = csv['Lines'].max()
    return csv.loc[(csv['Lines'] == maxlines)]

def join_csv_merge(dfcsv, fileTo, path_old, path_older, list_choice=[], limit=0):
    newCsv = pd.DataFrame()
    define_data = False
    lenght = len(list_choice)
    
    for i, row_merge in merge.iterrows():
        name = os.path.splitext(row_merge.File)[0]
        if (lenght > 0 and name not in list_choice):
            continue
            
        fe1 = pd.read_csv(path_older+'/'+row_merge.File)
        fe2 = pd.read_csv(path_old+'/'+row_merge.File)
        dates=[]

        if (define_data == False):
            for i, row in fe1.iterrows():
                dates.append(row['Data'])
            for i, row in fe2.iterrows():
                dates.append(row['Data'])
            newCsv['Data'] = dates
            define_data = True

        rows=[]
        for i, row in fe1.iterrows():
            rows.append(row['Fechamento'])
        for i, row in fe2.iterrows():
            rows.append(row['Fechamento'])
        
        newCsv[name] = rows
    
    if (limit > 0):
        # orderna do mais recente para o mais antigo
        pos = newCsv.shape[0]-limit
        newCsv = newCsv[pos:]
    newCsv.to_csv(fileTo,index=False)

    print('New file CSV successfully created')

# De 1 diretório
#### Usando a função para juntar os csv's de 'datasets-b3' em um único arquivo com o fechamento de todos

In [83]:
join_csv('datasets-b3', 'datasets/data.csv', limit=300)

New file CSV successfully created


#### Usando a função para juntar os csv's de 'datasets-alpha'

In [7]:
path_data_alpha = 'datasets/data_alpha.csv'
join_csv('historic-alpha vantage/datasets-treated', path_data_alpha, field='4. close', fielddate='date', limit=300)

New file CSV successfully created


Check NaN

In [10]:
data_alpha = pd.read_csv(path_data_alpha)
test=[]
for col in data_alpha.columns:
    su = data_alpha[col].isnull().sum()
    if(su > 0):
        test.append([col, su])
test

[]

In [9]:
del data_alpha['NTCO3']
data_alpha.to_csv(path_data_alpha, index=False)

# De 2 diretórios
#### Junta o preço de Fechamento de todos os csv de 2 diretório em um único csv, depois organiza todo os fechamentos em um csv
Somente os csv que existem nos dois diretórios

In [3]:
csv19 = find_csv_max_lines('datasets-b3-2019')
csv20 = find_csv_max_lines('datasets-b3')
csv20.shape, csv19.shape

((262, 2), (204, 2))

In [4]:
merge = pd.merge(csv20, csv19, left_on='File', right_on='File')
merge.shape

(200, 3)

### De todas as ações

In [5]:
#join_csv_merge(merge, 'datasets/data.csv', 'datasets-b3', 'datasets-b3-2019', limit=300)

### Somente de algumas ações

In [6]:
tickers = pd.read_csv('datasets/composicao_indice.csv', names=['Código'])['Código']
tickers

0     ABEV3
1     AZUL4
2     B3SA3
3     BBAS3
4     BBDC3
      ...  
70    VALE3
71    VIVT4
72    VVAR3
73    WEGE3
74    YDUQ3
Name: Código, Length: 75, dtype: object

In [7]:
join_csv_merge(merge, 'datasets/data_cart.csv', 'datasets-b3', 'datasets-b3-2019', tickers.tolist(), limit=300)

New file CSV successfully created


In [8]:
data = pd.read_csv('datasets/data_cart.csv', index_col=0)
data.shape, 'from {} to {}'.format(data.index[0], data.index[-1])

((300, 71), 'from 12/04/2019 to 26/06/2020')